In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import csv
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [2]:
img_width, img_height = 150, 150

train_data_dir = 'CelebA_Spoof/Data/train'
validation_data_dir = 'CelebA_Spoof/Data/test'
nb_train_samples = 1052
nb_validation_samples = 200
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',f1_m,precision_m, recall_m])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


model.save_weights('first_try.h5')

Found 1052 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
65/65 [==============================] - 62s 953ms/step - loss: 0.5246 - accuracy: 0.7471 - f1_m: 0.6853 - precision_m: 0.8510 - recall_m: 0.6265 - val_loss: 0.6122 - val_accuracy: 0.5938 - val_f1_m: 0.3179 - val_precision_m: 0.7222 - val_recall_m: 0.2067
Epoch 2/50
65/65 [==============================] - 58s 888ms/step - loss: 0.3156 - accuracy: 0.8600 - f1_m: 0.8428 - precision_m: 0.9153 - recall_m: 0.8171 - val_loss: 0.6742 - val_accuracy: 0.6510 - val_f1_m: 0.4564 - val_precision_m: 0.9236 - val_recall_m: 0.3450
Epoch 3/50
65/65 [==============================] - 58s 897ms/step - loss: 0.2796 - accuracy: 0.8764 - f1_m: 0.8627 - precision_m: 0.9023 - recall_m: 0.8526 - val_loss: 0.4703 - val_accuracy: 0.7708 - val_f1_m: 0.7602 - val_precision_m: 0.7442 - val_recall_m: 0.7930
Epoch 4/50
65/65 [======================

65/65 [==============================] - 57s 879ms/step - loss: 0.1110 - accuracy: 0.9614 - f1_m: 0.9577 - precision_m: 0.9656 - recall_m: 0.9567 - val_loss: 0.3017 - val_accuracy: 0.8594 - val_f1_m: 0.8585 - val_precision_m: 0.7870 - val_recall_m: 0.9716
Epoch 31/50
65/65 [==============================] - 52s 802ms/step - loss: 0.2955 - accuracy: 0.9672 - f1_m: 0.9652 - precision_m: 0.9665 - recall_m: 0.9688 - val_loss: 0.4788 - val_accuracy: 0.9010 - val_f1_m: 0.8943 - val_precision_m: 0.9461 - val_recall_m: 0.8608
Epoch 32/50
65/65 [==============================] - 47s 717ms/step - loss: 0.1103 - accuracy: 0.9595 - f1_m: 0.9581 - precision_m: 0.9612 - recall_m: 0.9615 - val_loss: 0.7792 - val_accuracy: 0.8542 - val_f1_m: 0.8223 - val_precision_m: 0.9708 - val_recall_m: 0.7378
Epoch 33/50
65/65 [==============================] - 47s 717ms/step - loss: 0.1634 - accuracy: 0.9604 - f1_m: 0.9594 - precision_m: 0.9605 - recall_m: 0.9642 - val_loss: 0.3903 - val_accuracy: 0.8854 - val_f1

In [3]:
model.evaluate_generator(
  validation_generator,
  nb_validation_samples/batch_size,
  workers = 1
)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.3410029113292694,
 0.8849999904632568,
 0.868285059928894,
 0.9415751099586487,
 0.8213675618171692]